Iniciaremos trabajando con el archivo: **car.data**

In [1]:
#Importamos las librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_car = pd.read_csv("C:/Users/Layla/Desktop/Docencia/Coder House/Materiales complementarios/Mod 6 - Data Science/3. Mejora de Modelos de Machine Learning II/Datos/car.data", sep = ",", header=None)
df_car.head()

,0,1,2,3,4,5,6
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [2]:
#Dimension del dataset
print(df_car.shape)

(1728, 7)


In [3]:
#Tipo de dato para cada columna
df_car.dtypes

0    object
1    object
2    object
3    object
4    object
5    object
6    object
dtype: object

In [4]:
#Información del dataset
df_car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1728 non-null   object
 1   1       1728 non-null   object
 2   2       1728 non-null   object
 3   3       1728 non-null   object
 4   4       1728 non-null   object
 5   5       1728 non-null   object
 6   6       1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [5]:
#Renombramos las columnas para lograr una mayor interpretacion
df_car.rename(columns={0:'x0', 1:'x1',2:'x2',3:'x3',4:'x4',5:'x5',6:'y'}, inplace=True)

In [6]:
#Verificamos el cambio realizado
df_car.columns

Index(['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'y'], dtype='object')

In [7]:
#Separacion en X e y
y = df_car['y'].to_numpy()

for n in df_car.columns:
  if str(df_car[n].dtype) == 'object' or str(df_car[n].dtype) == 'category':
    df_car[n] = df_car[n].astype('category').cat.codes
X = df_car.drop(['y'], axis=1).to_numpy()
df_car.head()

,x0,x1,x2,x3,x4,x5,y
0,3,3,0,0,2,1,2
1,3,3,0,0,2,2,2
2,3,3,0,0,2,0,2
3,3,3,0,0,1,1,2
4,3,3,0,0,1,2,2


Iniciaremos aplicando el método de **StratifiedKFold** para posteriormente comparar los resultados obtenidos con **KFold**. 

In [8]:
#StratifiedKFold - Train y Test
from sklearn.model_selection import StratifiedKFold
skf_car = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
train, test = list(skf_car.split(X, y))[0]
X_train = X[train]
X_test = X[test]
y_train = y[train]
y_test= y[test]

In [9]:
#Entrenar arbol y encontrar el mejor alpha con GridSearchCV y StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

t_car = DecisionTreeClassifier()
par_car = list(np.arange(0.0, 1., step=0.05))
cv_car = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
tunner_car = GridSearchCV(estimator=t_car, param_grid={'ccp_alpha':par_car}, cv=cv_car)
_=tunner_car.fit(X_train, y_train)

In [10]:
print('Train score StratifiedKFold: ' + str(tunner_car.score(X_train, y_train)))
print('Test score StratifiedKFold: ' + str(tunner_car.score(X_test, y_test)))

Train score StratifiedKFold: 1.0
Test score StratifiedKFold: 0.976878612716763


In [11]:
#KFold
from sklearn.model_selection import KFold
KFold_car = KFold(n_splits=5, random_state=0, shuffle=True)
train_2, test_2 = list(KFold_car.split(X))[0]
X_train_2 = X[train_2]
X_test_2 = X[test_2]
y_train_2 = y[train_2]
y_test_2 = y[test_2]

In [12]:
#Entrenar arbol y encontrar el mejor alpha con GridSearchCV y KFold
t_car2 = DecisionTreeClassifier()
par_car2 = list(np.arange(0.0, 1., step=0.05))
cv_car2 = KFold(n_splits=5, random_state=0, shuffle=True)
tunner_car2 = GridSearchCV(estimator=t_car2, param_grid={'ccp_alpha':par_car2}, cv=cv_car2)
_=tunner_car2.fit(X_train_2, y_train_2)

In [13]:
#Metricas
print('Train score KFold: ' + str(tunner_car2.score(X_train_2, y_train_2)))
print('Test score KFold: ' + str(tunner_car2.score(X_test_2, y_test_2)))

Train score KFold: 1.0
Test score KFold: 0.9826589595375722


**Conclusiones e Interpretaciones del dataset: car.data**

Si comenzamos nuestro análisis sobre el método de StratifiedKFold, se observan los siguientes valores para las métricas obtenidas:

* Train score StratifiedKFold: 1.0
* Test score StratifiedKFold: 0.9826589595375722

Claramente, podemos identificar que nuestro modelo se está "sobreajustando", razón inicial que podría deberse al pequeño tamaño del dataset 1728 registros respectivamente.

Ahora bien, analizando el método de KFold, los resultados de Train y Test se detallan a continuación: 

* Train score KFold: 1.0
* Test score KFold: 0.9682080924855492

Para este caso en particular para las métricas obtenidas, no se observan grandes mejoras al implementar este tipo de método de CrossValidation, dado que el algoritmo aun persiste sobreajustado y el Test Score se ve reducido muy levemente. 

Continuaremos trabajando con el archivo: **aug_train**

**Consignas**:
    
Se deberá replicar el ejemplo anterior realizado, sobre el dataset propuesto para el método de StratifiedKFold y KFold.
El algoritmo a entrenar será un DecisionTreeClassifier. Finalmente se solicita elaborar una pequeña conclusión e interpretación de las aplicaciones realizadas.

Aclaración: La variable de interes es "target".

In [16]:
#Importamos el segundo data set
df_aug = pd.read_csv("C:/Users/Layla/Desktop/Docencia/Coder House/Materiales complementarios/Mod 6 - Data Science/3. Mejora de Modelos de Machine Learning II/Datos/aug_train.csv", sep = ",")
df_aug

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,7386,city_173,0.878,Male,No relevent experience,no_enrollment,Graduate,Humanities,14,NaN,NaN,1,42,1.0
19154,31398,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,NaN,NaN,4,52,1.0
19155,24576,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,4,44,0.0
19156,5756,city_65,0.802,Male,Has relevent experience,no_enrollment,High School,NaN,<1,500-999,Pvt Ltd,2,97,0.0


In [29]:
#Dimension del dataset

    ###Completar

In [30]:
#Tipo de dato para cada columna

  ###Completar

In [31]:
#Información del dataset

  ###Completar

In [32]:
#Separacin en X e y

  ###Completar

In [33]:
#Dividir Train y Test para CV - StratifiedKFold

  ###Completar

In [34]:
#Entrenar arbol y encontrar el mejor alpha con GridSearchCV y StratifiedKFold
  
  ###Completar

In [35]:
#print('Train score StratifiedKFold: ' + str(tunner_aug.score(X_train, y_train)))
#print('Test score StratifiedKFold: ' + str(tunner_aug.score(X_test, y_test)))

In [36]:
#KFold

  ###Completar

In [38]:
#Entrenar arbol y encontrar el mejor alpha con GridSearchCV y KFold
  
  ###Completar

In [39]:
#Metricas

  ###Completar

**Conclusionees e Interpretaciones del dataset: aug_train**

  ###Completar